1. tokenizer，构造输入

In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

- tokenizer, model:相匹配， tokenizer output -> model input
- tokenizer原理: vacab中 token与id一一对应
    - len(input_ids) == len(attn_masks)
    - attn_masks与padding相匹配

In [2]:
test_sentences = ['today is not that bad', 'today is so bad']
model_name = 'distilbert-base-uncased-finetuned-sst-2-english'

In [3]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

In [4]:
batch_input = tokenizer(test_sentences, truncation=True, padding=True, return_tensors='pt') # 返回torch.tensor
batch_input

{'input_ids': tensor([[ 101, 2651, 2003, 2025, 2008, 2919,  102],
        [ 101, 2651, 2003, 2061, 2919,  102,    0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 0]])}

In [5]:
# tokenizer(test_sentences[0],)
# tokenizer.encode(test_sentences[0])
tokenizer.decode([101, 2651, 2003, 2025, 2008, 2919, 102]) # '[CLS] today is not that bad [SEP]'
# tokenizer.tokenize(test_sentences[0])
# tokenizer.convert_tokens_to_ids(tokenizer.tokenize(test_sentences[0]))

'[CLS] today is not that bad [SEP]'

2. model,调用模型

In [6]:
import torch
# import torch.nn as nn
from torch.nn import *
# import torch.nn.functional as F
# from torch.nn import functional as F

In [10]:
model.config

DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased-finetuned-sst-2-english",
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "finetuning_task": "sst-2",
  "hidden_dim": 3072,
  "id2label": {
    "0": "NEGATIVE",
    "1": "POSITIVE"
  },
  "initializer_range": 0.02,
  "label2id": {
    "NEGATIVE": 0,
    "POSITIVE": 1
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.37.2",
  "vocab_size": 30522
}

In [11]:
with torch.no_grad():
    outputs = model(**batch_input)
    print(outputs)
    scores = torch.nn.functional.softmax(outputs.logits, dim=1)
    print(scores)
    labels = torch.argmax(scores, dim=1)
    print(labels)
    labels = [model.config.id2label[id] for id in labels.tolist()]
    print(labels)

SequenceClassifierOutput(loss=None, logits=tensor([[-3.4620,  3.6118],
        [ 4.7508, -3.7899]]), hidden_states=None, attentions=None)
tensor([[8.4632e-04, 9.9915e-01],
        [9.9980e-01, 1.9531e-04]])
tensor([1, 0])
['POSITIVE', 'NEGATIVE']
